In [ ]:
import collections
import sklearn.model_selection

from classes import *
from files import create_outfile_each_fold, create_outfile_mean, get_path_outfile_each_fold, get_path_mean, load_image, load_mask, save_figs
from gpu import set_avx_avx2, set_gpu
from metrics import calculate_iou_dice, dice_loss, jaccard_distance_loss
from model import cfg_model, train_model

In [11]:
tensorflow.__version__

'2.8.2'

# GPU

In [12]:
set_avx_avx2()
set_gpu()

GPU: /physical_device:GPU:0


In [13]:
cfg = {
    "batch_size": 4,
    "cross_validation": 2,
    "epochs": 4,
    "image_size": 400,
    "learning_rate": 0.05,
    "random_state": 1234,
    "test_size": 0.15,
    "val_size": 0.05,
    "path_base": "dataset",
    "path_out": "out"
}

In [14]:
def get_index(cfg):
    return sklearn.model_selection.StratifiedShuffleSplit(n_splits=cfg["cross_validation"], test_size=cfg["test_size"], random_state=cfg["random_state"])

def get_list_index(cfg, x, y):
    list_index = get_index(cfg)
    return list([Index(fold, index_train, index_test) for fold, (index_train, index_test) in enumerate(list_index.split(x, y))])

In [15]:
piperaceae_dataset = Dataset(cfg, os.path.join(cfg["path_base"], "images"), os.path.join(cfg["path_base"], "mask"))
len(getattr(piperaceae_dataset, "list_images"))

375

In [16]:
list_images = getattr(piperaceae_dataset, "list_images")
x = numpy.array(list([l for l in list_images]), dtype=object)
y = numpy.array(list([getattr(l["image"], "label").value for l in list_images]))

collections.Counter(y.tolist())

Counter({'manekia': 75,
         'ottonia': 75,
         'peperomia': 75,
         'piper': 75,
         'pothomorphe': 75})

In [17]:
list_index = get_list_index(cfg, x, y)
len(list_index)

2

In [18]:
def get_image(cfg, list_images):
    x = list([])
    y = list([])
    for image in list_images:
        x.append(getattr(image["image"], "image"))
        y.append(getattr(image["mask"], "image"))
    x = numpy.array(x)
    y = numpy.array(y)
    return x.reshape(x.shape[0], cfg["image_size"], cfg["image_size"], 1), y.reshape(y.shape[0], cfg["image_size"], cfg["image_size"], 1)

list_result = list([])
path = get_path_mean(cfg)
pathlib.Path(path).mkdir(parents=True, exist_ok=True)
for index in list_index:
    fold = str(getattr(index, "fold"))
    print(fold)
    path_outfile_each_fold = get_path_outfile_each_fold(fold, path)
    pathlib.Path(path_outfile_each_fold).mkdir(parents=True, exist_ok=True)

    x_train, y_train = x[getattr(index, "index_train")], y[getattr(index, "index_train")]
    x_test, y_test = x[getattr(index, "index_test")], y[getattr(index, "index_test")]
    x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(x_train, y_train, test_size=cfg["val_size"], random_state=cfg["random_state"])

    x_train, y_train = get_image(cfg, x_train)
    x_test, y_test = get_image(cfg, x_test)
    x_val, y_val = get_image(cfg, x_val)

    print(x_train.shape, y_train.shape)
    print(x_test.shape, y_test.shape)
    print(x_val.shape, y_val.shape)

    # unet_filename = "batch16+lr0_05+epoch200+steps22+unet.h5"
    checkpointer, reduce_learning_rate, steps_per_epoch, strategy, train_generator, unet_filename = cfg_model(cfg, fold, path_outfile_each_fold, x_train, y_train)

    if os.path.exists(unet_filename):
        model = tensorflow.keras.models.load_model(unet_filename, custom_objects = {"jaccard_distance_loss": jaccard_distance_loss,"dice_coef": dice_coef})
    else:
        elapsed_time, model = train_model(cfg, checkpointer, fold, path_outfile_each_fold, reduce_learning_rate, steps_per_epoch, strategy, train_generator, x_val, y_val)

    metrics = calculate_iou_dice(model, x_test, x_train, x_val, y_test, y_train, y_val)
    list_result.append(metrics)
    create_outfile_each_fold(fold, metrics, path_outfile_each_fold)
    # save_figs(cfg, model, path_outfile_each_fold, x, x_train, x_val, y_test, y_train, y_val)

0
(302, 400, 400, 1) (302, 400, 400, 1)
(57, 400, 400, 1) (57, 400, 400, 1)
(16, 400, 400, 1) (16, 400, 400, 1)


C:\Users\xandao\Documents\GitHub\piperaceae-segmentation\files.py:92: UserWarning: out\26-07-2022-09-05-13\0\test\0\0manekiaCRI006138+mask_unet.png is a low contrast image
  skimage.io.imsave(filename, skimage.img_as_ubyte(image))


ValueError: Image must be 2D (grayscale, RGB, or RGBA).

In [ ]:
create_outfile_mean(cfg, list_result, path)
len(list_result)